In [9]:
import json

In [4]:
places = ["hotel", "park", "restaurant", "shop"]

In [30]:
unique_types = []
for place in places:
    with open(f"{place}_data.json", "r") as f:
        data = json.load(f)
        results = data["results"]
        for result in results:
            types = result["types"]
            for type in types:
                if type not in unique_types:
                    unique_types.append(type)

NameError: name 'places' is not defined

In [10]:
with open("unique_types.txt", "w") as f:
    for type in unique_types:
        f.write(type + "\n")

In [34]:
unique_types = []
with open("unique_types.txt", "r") as f:
    for type in f.readlines():
        unique_types.append(type.strip())

In [7]:
import googlemaps
import config

google = googlemaps.Client(config.API_KEY)

In [14]:
with open("unique_types.txt", "r") as f:
    for unique_type in f.readlines():
        unique_type = unique_type.strip()
        result = google.places(query=unique_type)
        with open(f"places/{unique_type}_data.json", "w") as out:
            json.dump(result, out, indent=4)

In [56]:
import pandas as pd

In [ ]:
mapping = {
    "lodging": 0,
    "establishment": 1,
    "park": 2,
    "tourist_attraction": 3,
    "restaurant": 4,
    "food": 5,
    "grocery_or_supermarket": 6,
    "store": 7,
    "meal_takeaway": 8,
    "cafe": 9,
    "department_store": 10,
    "convenience_store": 11,
    "clothing_store": 12,
    "shoe_store": 13,
    "shopping_mall": 14,
    "book_store": 15,
    "electronics_store": 16,
    "health": 17,
    "bakery": 18,
    "home_goods_store": 19
}

In [52]:
def vectorize(location: dict):
    """
    Given a location from the Google Places API, return the location
    as a feature vector.
    """
    result = {
        "address": location["formatted_address"],
        "name": location["name"],
        "lat": location["geometry"]["location"]["lat"],
        "lng": location["geometry"]["location"]["lng"],
        "place_id": location["place_id"],
        "rating": location["rating"],
        "user_ratings_total": location["user_ratings_total"],
    }
    if "photos" in location.keys():
        result["html_attributions"] = location["photos"][0]["html_attributions"][0]
        result["photo_reference"] = location["photos"][0]["photo_reference"]
    else:
        result["html_attributions"] = "Nil"
        result["photo_reference"] = "Nil"
    result.update({t: 1 if t in location["types"] else 0 for t in unique_types})
    return result

In [54]:
df = []
for t in unique_types:
    print(t)
    with open(f"places/{t}_data.json", "r") as f:
        data = json.load(f)
        data = data["results"]
        for location in data:
            df.append(vectorize(location))
print(len(df))

lodging
point_of_interest
establishment
park
tourist_attraction
restaurant
food
grocery_or_supermarket
store
meal_takeaway
cafe
department_store
convenience_store
clothing_store
shoe_store
shopping_mall
book_store
electronics_store
health
bakery
home_goods_store
244


In [57]:
df = pd.DataFrame(df)

In [59]:
df.to_csv("data.csv")